<a href="https://colab.research.google.com/github/ibitoladgr8/Machine-Learning/blob/main/MENTAL_HEALTH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')

test_df.shape

(93800, 19)

In [ ]:
def profession_noise(df):

    profession_counts = df['Profession'].value_counts()

    # Create a list of profession with count less than 100
    profession_to_replace = profession_counts[profession_counts < 100].index

    # Replace with counts less than 100 with 'Other'
    df['Profession'] = df['Profession'].apply(lambda x: 'Unknown_Profession' if x in profession_to_replace else x)

    return df

def city_noise(df):
    # Get the value counts for the 'City' column
    city_counts = df['City'].value_counts()

    # Create a list of cities with count less than 100
    cities_to_replace = city_counts[city_counts < 100].index

    # Replace cities with counts less than 100 with 'Other'
    df['City'] = df['City'].apply(lambda x: 'Other_Cities' if x in cities_to_replace else x)

    return df

valid_categories = ['Moderate', 'Unhealthy', 'Healthy']

def diet_noise(train):

    # Replace invalid categories as nan
    train['Dietary Habits'] = train['Dietary Habits'].apply(
        lambda x: x if x in valid_categories else np.nan)

    return train


In [ ]:
def noise(df):
    return diet_noise(profession_noise(city_noise(df)))
train_df = noise(train_df)
#test_df = noise(test_df)

test_df.shape

(93800, 19)

In [ ]:
train_df.dropna(subset=['Depression'], inplace=True)
train_df.dtypes

,0
id,int64
Name,object
Gender,object
Age,float64
City,object
Working Professional or Student,object
Profession,object
Academic Pressure,float64
Work Pressure,float64
CGPA,float64


In [ ]:
from sklearn.model_selection import train_test_split

X = train_df.drop(['id','Name','Depression'], axis=1)
y = train_df['Depression']

In [ ]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 100 and
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('num', SimpleImputer(strategy='constant'),
                                 ['Age', 'Academic Pressure', 'Work Pressure',
                                  'CGPA', 'Study Satisfaction',
                                  'Job Satisfaction', 'Work/Study Hours',
                                  'Financial Stress']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Gender', 'City',
                                  'Working Professional or Student',
                                  'Profession', 'Sleep Duration',
                                  'Dietary Habits', 'Degree',
                                  'Have you ever had suicidal thoughts ?',
                                  'Family History of Mental Illness'])])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2


In [ ]:
feature_names = preprocessor.get_feature_names_out()

# Calculate the number of features after preprocessing
num_features_after_preprocessing = len(feature_names)

# Update the input shape of your model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=[num_features_after_preprocessing]), # Updated input shape
    tf.keras.layers.Dropout(0.2),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Re-compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
from sklearn.metrics import mean_absolute_error

early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model
my_pipeline.named_steps['model'].fit(
    my_pipeline.named_steps['preprocessor'].transform(X_train),
    y_train,
    batch_size=512,  # Add batch size
    epochs=1000,      # Add epochs
    callbacks=[early_stopping],  # Add callbacks
    verbose=0         # Add verbose
)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

226/226 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
MAE: 0.07792901336171375


In [ ]:
test_X = test_df[my_cols].copy()
predictions = my_pipeline.predict(test_df)

output = pd.DataFrame({'id': test_df['id'], 'Depression': predictions.flatten()})
output.to_csv('submission.csv', index=False)

2932/2932 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step


In [ ]:
submission = pd.read_csv('/content/submission.csv')
submission.head()

,id,Depression
0,140700,3.081987e-11
1,140701,8.556272e-13
2,140702,1.550044e-19
3,140703,9.994337e-01
4,140704,2.941189e-10


In [ ]:
def convert_submission(x): # Changed input to 'x' instead of 'df'
  if x >= 0.5: # Access the value directly using 'x'
    return 1
  else:
    return 0

submission['Depression'] = submission['Depression'].apply(convert_submission) # Pass the function directly
submission.to_csv('submission.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model1=RandomForestClassifier(n_estimators=100, random_state=42)
my_pipeline1 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model1)
                             ])

# Preprocessing of training data, fit model
#my_pipeline.named_steps['model'].fit(my_pipeline.named_steps['preprocessor'].transform(X_train), y_train, epochs=10)
# Preprocessing of validation data, get predictions
my_pipeline1.fit(X_train, y_train)
preds1 = my_pipeline1.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

predictions1=my_pipeline1.predict(test_df)
output = pd.DataFrame({'id': test_df['id'], 'Depression': predictions1.flatten()})
output.to_csv('submissionrf.csv', index=False)

MAE: 0.08655799680709937


In [ ]:
from xgboost import XGBClassifier

model2= XGBClassifier(n_estimators=100,
                      random_state=42,
                      learning_rate=0.05,
                      max_depth=3,
                      subsample=0.8,
                      colsample_bytree=0.8
                      )
my_pipeline2 = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model2)
                             ])

# Preprocessing of training data, fit model
#my_pipeline.named_steps['model'].fit(my_pipeline.named_steps['preprocessor'].transform(X_train), y_train, epochs=10)
# Preprocessing of validation data, get predictions
my_pipeline2.fit(X_train, y_train)
preds2 = my_pipeline2.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

predictions2=my_pipeline2.predict(test_df)
output = pd.DataFrame({'id': test_df['id'], 'Depression': predictions2.flatten()})
output.to_csv('submissionxgb.csv', index=False)

MAE: 0.07792901336171375
